In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

base_src = '../data/drive-download-20240102T142504Z-001'
u_user_src = os.path.join(base_src, 'u.user')
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(
    u_user_src,
    sep='|',
    names=u_cols,
    encoding='latin-1'
)
users = users.set_index('user_id')

u_item_src = os.path.join(base_src, 'u.item')
i_cols = [
    'movie_id', 'title', 'release date', 'video release date',
    'IMDB URL', 'unknown', 'Action', 'Adventure', 'Animat ion',
    'Children\'s', 'Comedy', 'Crime', 'Documentary ', 'Drama',
    'Fantasy', 'Film- Noir', 'Horror', 'Musical', 'Mystery',
    'Romance ', 'Sci-Fi', 'Thriller', 'War', 'Western'
]
movies = pd.read_csv(
    u_item_src,
    sep='|',
    names=i_cols,
    encoding='latin-1'
)
movies = movies.set_index('movie_id')

u_data_src = os.path.join(base_src, 'u.data')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(
    u_data_src,
    sep='\t',
    names=r_cols,
    encoding='latin-1'
)


# 정확도 (RMSE)를 계산하는 함수
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred)) ** 2))


# 모델별 RMSE를 계산하는 함수
def score(model, neighbor_size=0):
    # 테스트 데이터의 user_id와 movie)id 간 pair를 맞춰 튜플형 원소 리스트 데이터 생성
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    # 모든 사용자 - 영화 pair에 대해서 주어진 예측 모델에 의해 예측값 계산 및 리스트형 데이터 생성
    y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
    # 실제 평점값
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)


# 데이터셋 생성
x = ratings.copy()
y = ratings['user_id']

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, stratify=y
)

rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

# train set의 모든 가능한 사용자 pair의 Cosine similarities 계산
from sklearn.metrics.pairwise import cosine_similarity

# 코사인 유사도를 구하기 위해 rating값을 복사하고, 계산 시 NaN값 에러 대비 0으로 변환
matrix_dummy = rating_matrix.copy().fillna(0)
# 모든 사용자간 코사인 유사고 계산
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
# 필요한 값 조회를 위해 인덱스 및 컬럼명 지정
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

In [2]:
# Neighbor size를 정해서 예측치를 계산하는 함수
def CF_knn(user_id, movie_id, neighbor_size=0):
    # train/test set의 분할에 따라 생성한 rating_matrix에 movie_id가 있는지 확인
    if movie_id in rating_matrix.columns:
        # 주어진 사용자 (user_id)와 다른 사용자 간의 유사도 추출
        sim_scores = user_similarity[user_id].copy()
        # 주어진 영화 (movie_id)를 평가한 모든 사용자의 평점 정보를 추출
        movie_ratings = rating_matrix[movie_id].copy()
        # 주어진 영화를 평가하지 않은 사용자의 인덱스를 추출하고, 해당 인덱스의 유사도 및 평점 정보에서 제외
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        
        # 주어진 영화를 평가하지 않은 사용자 제외
        movie_ratings = movie_ratings.dropna()
        # 주어진 영화를 평가하지 않은 사용자의 유사도 제외
        sim_scores = sim_scores.drop(none_rating_idx)

        if neighbor_size == 0:  # simple 협업 필터링
            mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()

        else:
            if len(sim_scores) > 1:
                neighbor_size = min(neighbor_size, len(sim_scores))
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                
                # 유사도(sim_scores)를 순서대로 오름차순 정렬한 후 인덱스(user_id)를 추출
                user_idx = np.argsort(sim_scores)
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                
                mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            else:
                mean_rating = 3.0
    else:
        mean_rating = 3.0
    return mean_rating

        
        

In [3]:
# neighbor sizer가 10,20,30,40,50,60 인 경우에 대해서 RMSE를 계산하고 이를 출력 
for neighbor_size in [10, 20, 30, 40, 50, 60]:
    print('Neighbor size = %d : RMSE = %.4f' % (neighbor_size, score(CF_knn, neighbor_size)))

Neighbor size = 10 : RMSE = 1.0269
Neighbor size = 20 : RMSE = 1.0145
Neighbor size = 30 : RMSE = 1.0127
Neighbor size = 40 : RMSE = 1.0126
Neighbor size = 50 : RMSE = 1.0131
Neighbor size = 60 : RMSE = 1.0136
